In [1]:
import pandas as pd
import numpy as np
import json
import shapely.geometry
import shapely
import uuid
import pickle

In [6]:
points = [
[35.0424114,	-89.9766792],
[35.0237222,	-90.7479444],
[35.0242222,	-90.8151111],
[35.0245000,	-90.8556944],
[35.0000000,	-92.0000000],
[35.5000000,	-96.0000000],
[36.0000000,	-98.0000000],
[36.5000000,	-102.0000000],
[36.5000000,	-106.0000000],
[36.0000000,	-110.0000000],
[35.7338139,	-112.1087333],
[35.6047167,	-112.3833333],
[35.0000000,	-116.0000000],
[34.4224389,	-118.0258528],
[34.0000000,	-120.0000000],
[34.2489750,	-123.0902667],
[34.0230556,	-134.3461111],
[31.1783333,	-141.2900000],
[27.8922222,	-147.9700000],
[27.8922222,	-147.9700000],
[24.2958333,	-154.2500000],
[23.5280306,	-155.4788361],
[23.5280278,	-155.4788333],
[22.3215000,	-156.3425000],
[21.8405278,	-156.9656111],
[21.6684722,	-157.1872500],
[21.4165556,	-157.5100000],
[21.3276944,	-157.6233611],
[21.2651111,	-157.7029722],
[21.3178172,	-157.9202275]]
for point in points:
    point.reverse()
ls = shapely.geometry.LineString(points)
gjson = shapely.to_geojson(ls)
gjson

'{"type":"LineString","coordinates":[[-89.9766792,35.0424114],[-90.7479444,35.0237222],[-90.8151111,35.0242222],[-90.8556944,35.0245],[-92.0,35.0],[-96.0,35.5],[-98.0,36.0],[-102.0,36.5],[-106.0,36.5],[-110.0,36.0],[-112.1087333,35.7338139],[-112.3833333,35.6047167],[-116.0,35.0],[-118.0258528,34.4224389],[-120.0,34.0],[-123.0902667,34.248975],[-134.3461111,34.0230556],[-141.29,31.1783333],[-147.97,27.8922222],[-147.97,27.8922222],[-154.25,24.2958333],[-155.4788361,23.5280306],[-155.4788333,23.5280278],[-156.3425,22.3215],[-156.9656111,21.8405278],[-157.18725,21.6684722],[-157.51,21.4165556],[-157.6233611,21.3276944],[-157.7029722,21.2651111],[-157.9202275,21.3178172]]}'

In [62]:
df = pd.read_csv('../data/my_props_4.csv')

In [63]:
group = df.groupby('region_id')
group

In [88]:
neighborhoods = []
for g in group:
  gdf = g[1]
  gdf['coords'] = gdf.apply(lambda x: [x['longitude'], x['latitude']], axis=1)
  multipoint = shapely.geometry.MultiPoint(gdf['coords'].tolist())
  neighborhood = {
    'id': str(uuid.uuid4()),
    'region_id': g[0],
    'multipoint': json.loads(shapely.to_geojson(multipoint)),
    'centroid': [multipoint.centroid.x, multipoint.centroid.y],
  }
  neighborhoods.append(neighborhood)
# save neighborhoods to json file
with open('../data/final/neighborhoods.json', 'w') as f:
  json.dump(neighborhoods, f)

In [76]:
with open('../data/final/neighborhoods.pkl', 'wb') as handle:
    pickle.dump(neighborhoods, handle)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [57]:
df.loc[df['region_id'] == 'cluster_23_atlanta', 'region_id'] = 'nan_chamblee'

In [48]:
df.loc[df['region_id'] == 'nan_doraville']

,propertyId,address_city,address_county,address_zip,latitude,longitude,neighborhood,found_city,accepted_city,region_id
62,14571062,atlanta,dekalb,30340,33.891880,-84.247795,cluster_-1,NaN,atlanta,nan_doraville
113,14574812,atlanta,dekalb,30340,33.910070,-84.259940,NaN,doraville,doraville,nan_doraville
165,14571547,atlanta,dekalb,30341,33.886307,-84.250340,cluster_30,NaN,atlanta,nan_doraville
200,14578876,doraville,dekalb,30340,33.915478,-84.257380,NaN,doraville,doraville,nan_doraville
283,294843044,atlanta,dekalb,30340,33.889030,-84.276070,NaN,doraville,doraville,nan_doraville
...,...,...,...,...,...,...,...,...,...,...
352591,14578471,doraville,dekalb,30360,33.913044,-84.280940,NaN,doraville,doraville,nan_doraville
352716,14571170,atlanta,dekalb,30340,33.893448,-84.247520,cluster_-1,NaN,atlanta,nan_doraville
352805,14571634,atlanta,dekalb,30340,33.893740,-84.269690,cluster_-1,NaN,atlanta,nan_doraville
352886,14574007,atlanta,dekalb,30340,33.898480,-84.272840,NaN,doraville,doraville,nan_doraville


In [37]:
def conform_cluster_to_city(df, clusters, city):
  df.loc[df['neighborhood'].isin(clusters), 'found_city'] = city
  df.loc[df['neighborhood'].isin(clusters), 'accepted_city'] = city
  df.loc[df['neighborhood'].isin(clusters), 'neighborhood'] = 'NaN'
  df.loc[df['neighborhood'].isin(clusters), 'region_id'] = f'nan_{city}'
  return df

In [41]:
df = conform_cluster_to_city(df, ['cluster_30', 'cluster_32', 'cluster_6', 'cluster_7'], 'doraville')
df = conform_cluster_to_city(df, ['cluster_23'], 'chamblee')

In [68]:
df.to_csv('../data/final/properties.csv', index=False)